In [134]:
import numpy as np
import pandas as pd
import config # a python file that contains path to TIMSS data files

#from sklearn import tree
#from sklearn import BaggingClassifier
#from sklearn import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ml_data_1 = pd.read_csv(config.ml_data_1, index_col=False)

In [6]:
x = ml_data_1.drop(columns='mean_PV')
x.sample()

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I
2022,No,Enough to fill two bookcases (101–200 books),Agree a lot,Agree a lot,At least once a week,Agree a little,Agree a lot,Every or almost every lesson,Agree a lot,Agree a lot


In [7]:
y = ml_data_1.mean_PV
y.sample()

625    503.206364
Name: mean_PV, dtype: float64

In [8]:
from kmodes.kmodes import KModes

# Instantiate the k-modes model
km = KModes(n_clusters=5, init='Huang', n_init=4, verbose=2, n_jobs=2)

# Fit the model to your categorical data
clusters = km.fit_predict(x)


Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initializing clusters
Starting iterations...
Starting iterations...
Run 1, iteration: 1/100, moves: 509, cost: 33186.0
Run 2, iteration: 1/100, moves: 3136, cost: 30471.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 2/100, moves: 948, cost: 32324.0
Run 3, iteration: 1/100, moves: 2416, cost: 30864.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 3/100, moves: 752, cost: 32324.0
Run 4, iteration: 1/100, moves: 2868, cost: 30215.0
Run 4, iteration: 2/100, moves: 1705, cost: 30215.0
Best run was number 4


In [9]:
clusters.max()

4

In [11]:
clusters

array([2, 2, 2, ..., 2, 0, 0], dtype=uint16)

Now that we have our clusters, we will add their numbers to each data point in our labels data in order to separate each cluster alone and build a separate model for each:

In [69]:
x['clusters'] = clusters
x['labels'] = y

In [61]:
x.describe()

,clusters,labels
count,8458.000000,8458.000000
mean,1.386616,385.328560
std,1.373483,62.364477
min,0.000000,194.209476
25%,0.000000,341.444794
50%,1.000000,379.916982
75%,2.000000,423.390834
max,4.000000,649.182768


In [62]:
x.sample(5)

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I,clusters,labels
5792,No,None or very few (0–10 books),Agree a lot,Agree a little,Once or twice a month,Agree a lot,Agree a lot,About half the lessons,Agree a little,Agree a lot,3,349.464136
788,No,Enough to fill three or more bookcases (more t...,Agree a little,Disagree a little,Never,Disagree a little,Disagree a lot,Every or almost every lesson,Agree a lot,Agree a lot,0,394.031688
7191,Yes,Enough to fill one shelf (11–25 books),Agree a little,Agree a little,At least once a week,Agree a little,Agree a little,Every or almost every lesson,Agree a little,Agree a lot,4,432.282972
7868,No,Enough to fill one shelf (11–25 books),Agree a lot,Agree a little,Never,Agree a little,Agree a lot,About half the lessons,Agree a little,Agree a little,1,336.710354
452,No,None or very few (0–10 books),Disagree a little,Agree a little,Once or twice a month,Disagree a little,Agree a little,Every or almost every lesson,Agree a little,Disagree a little,1,503.218330


Let's see what clusters do we have:

In [63]:
list_of_clusters = x['clusters'].unique()
list_of_clusters

array([2, 3, 0, 4, 1], dtype=uint16)

In [72]:
x[['labels','clusters']]

,labels,clusters
0,410.811348,2
1,433.181462,2
2,418.027694,2
3,467.300384,3
4,362.964464,2
...,...,...
8453,401.279696,0
8454,438.474212,4
8455,401.756130,2
8456,312.155188,0


Let's put each cluster in a different dataframe:

In [99]:
y_0 = x[['labels','clusters']].query('clusters==0')
x_0 = x.query('clusters==0')
y_1 = x[['labels','clusters']].query('clusters==1')
x_1 = x.query('clusters==1')
y_2 = x[['labels','clusters']].query('clusters==2')
x_2 = x.query('clusters==2')
y_3 = x[['labels','clusters']].query('clusters==3')
x_3 = x.query('clusters==3')
y_4 = x[['labels','clusters']].query('clusters==4')
x_4 = x.query('clusters==4')

In [81]:
x_3.count()

BSBG05D    886
BSBG04     886
BSBG13E    886
BSBG13D    886
BSBG14A    886
BSBM16A    886
BSBM17D    886
BSBM18C    886
BSBM20E    886
BSBM20I    886
labels     886
dtype: int64

Let's do some hyperparameters tuning of xgboost algorithm executed on "x_0" dataset using optuna:

In [55]:
import optuna
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [95]:
x_0.columns

Index(['BSBG05D', 'BSBG04', 'BSBG13E', 'BSBG13D', 'BSBG14A', 'BSBM16A',
       'BSBM17D', 'BSBM18C', 'BSBM20E', 'BSBM20I', 'clusters', 'labels'],
      dtype='object')

Let's keep only necessary features by deleting the 'clusters' column

In [100]:
df_list_features = [x_0, x_1, x_2, x_3, x_4]
df_list_labels = [y_0, y_1, y_3, y_4]
for df in df_list_features:
    del df['clusters']
    del df['labels']
for df in df_list_labels:
    del df['clusters']

In [101]:
x_0.columns

Index(['BSBG05D', 'BSBG04', 'BSBG13E', 'BSBG13D', 'BSBG14A', 'BSBM16A',
       'BSBM17D', 'BSBM18C', 'BSBM20E', 'BSBM20I'],
      dtype='object')

In [102]:
y_0.columns

Index(['labels'], dtype='object')

Let's start defining our objective function used by optuna, where we will define our parameters:

In [ ]:
#The following line is just to display, on hover, parameters of xgboost regressor

xgb.XGBRFRegressor()

In [103]:
x_0.dtypes

BSBG05D    object
BSBG04     object
BSBG13E    object
BSBG13D    object
BSBG14A    object
BSBM16A    object
BSBM17D    object
BSBM18C    object
BSBM20E    object
BSBM20I    object
dtype: object

In [112]:
x_0 = x_0.astype('category')
x_1 = x_1.astype('category')
x_2 = x_2.astype('category')
x_3 = x_3.astype('category')
x_4 = x_4.astype('category')

In [113]:
x_3.dtypes

BSBG05D    category
BSBG04     category
BSBG13E    category
BSBG13D    category
BSBG14A    category
BSBM16A    category
BSBM17D    category
BSBM18C    category
BSBM20E    category
BSBM20I    category
dtype: object

In [120]:
import category_encoders as ce
sklearn.set_config(transform_output="pandas")

encoder = ce.CatBoostEncoder(cols=['BSBG05D','BSBG04','BSBG13E','BSBG13D','BSBG14A','BSBM16A','BSBM17D','BSBM18C','BSBM20E','BSBM20I'])
encoder.fit(x_3, y_3)
X_cleaned = encoder.transform(x_3)


In [121]:
x_3.dtypes

BSBG05D    category
BSBG04     category
BSBG13E    category
BSBG13D    category
BSBG14A    category
BSBM16A    category
BSBM17D    category
BSBM18C    category
BSBM20E    category
BSBM20I    category
dtype: object

In [122]:
x_3.sample(1)

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I
528,No,None or very few (0–10 books),Agree a lot,Agree a lot,A few times a year,Agree a lot,Agree a lot,Never,Disagree a lot,Agree a lot


In [128]:
def objective( trail, data = x_3, target = y_3):
    train_x_3, test_x_3, train_y_3, test_y_3 = train_test_split(data,target, test_size=.20, random_state=30)
    param = {
        'tree_method':'hist',
        'enable_categorical':True,
        'lambda': trail.suggest_loguniform('lambda', 1e-10, 1e-4), # lambda uses L2 regularization
        'alpha': trail.suggest_loguniform('alpha', 1e-10, 1e-4),
        'colsample_bytree':trail.suggest_categorical('colsample_bytree', [.1,.2,.3,.4,.5,.8,.9]),
        'subsample':trail.suggest_categorical('subsample', [.1,.2,.3,.4,.5,.6,.7,.8,.9]),
        'learning_rate':trail.suggest_categorical('learning_rate', [.00001,.0003,.02,.004,.5,1,10]),
        'n_estimator': 30000,
        'max_depth':trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11]),
        'random_state': trail.suggest_categorical('random_state', [10,30,200,5000]),
        'min_child_weight': trail.suggest_int('min_child_weight',1,200),
        'verbosity': 3
    }
    xgb_x_3_model = xgb.XGBRFRegressor(**param)
    xgb_x_3_model.fit(train_x_3,train_y_3,eval_set = [(test_x_3,test_y_3)], verbose = True)
    prediction_xgb = xgb_x_3_model.predict(test_x_3)
    rmse = mean_squared_error(test_y_3,prediction_xgb)
    return rmse
    

In [131]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective, n_trials = 100)
find_param.best_params

[I 2023-11-06 18:05:12,573] A new study created in memory with name: no-name-002e7b9b-9996-41b9-aeb9-6cd0266e2264


[18:05:12] ======== Monitor (0): HostSketchContainer ========
[18:05:12] AllReduce: 6e-06s, 1 calls @ 6us

[18:05:12] MakeCuts: 1.5e-05s, 1 calls @ 15us

[18:05:12] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3
[0]	validation_0-rmse:62.74974


/tmp/ipykernel_13654/829001706.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trail.suggest_loguniform('lambda', 1e-10, 1e-4), # lambda uses L2 regularization
/tmp/ipykernel_13654/829001706.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trail.suggest_loguniform('alpha', 1e-10, 1e-4),
/home/aazzioui/Documents/WS/TIMSS-2019_student_score_prediction/env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:05:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2023-11-06 18:05:12,681] Trial 0 finished with value: 3937.5303993942935 and parameters: {'lambda'

[18:05:12] ======== Monitor (0): Learner ========
[18:05:12] Configure: 0.000842s, 1 calls @ 842us

[18:05:12] EvalOneIter: 0.000189s, 1 calls @ 189us

[18:05:12] GetGradient: 6e-06s, 1 calls @ 6us

[18:05:12] PredictRaw: 3e-06s, 1 calls @ 3us

[18:05:12] UpdateOneIter: 0.035939s, 1 calls @ 35939us

[18:05:12] ======== Monitor (0): GBTree ========
[18:05:12] BoostNewTrees: 0.035005s, 1 calls @ 35005us

[18:05:12] CommitModel: 3e-06s, 1 calls @ 3us

[18:05:12] ======== Monitor (0): HistUpdater ========
[18:05:12] BuildHistogram: 0.00123s, 49 calls @ 1230us

[18:05:12] EvaluateSplits: 0.010216s, 149 calls @ 10216us

[18:05:12] InitData: 0.000992s, 100 calls @ 992us

[18:05:12] InitRoot: 0.011405s, 100 calls @ 11405us

[18:05:12] LeafPartition: 1.5e-05s, 100 calls @ 15us

[18:05:12] UpdatePosition: 0.017811s, 49 calls @ 17811us

[18:05:12] UpdateTree: 0.032267s, 100 calls @ 32267us

[18:05:12] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3
[18:05:12] ======== Monitor (0): H

[I 2023-11-06 18:05:12,921] Trial 1 finished with value: 5912.444335313394 and parameters: {'lambda': 3.0679670173532456e-06, 'alpha': 9.994301187796778e-08, 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 10, 'max_depth': 5, 'random_state': 10, 'min_child_weight': 68}. Best is trial 0 with value: 3937.5303993942935.


...

/home/aazzioui/Documents/WS/TIMSS-2019_student_score_prediction/env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [18:06:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:200.57784


[I 2023-11-06 18:06:10,816] Trial 99 finished with value: 40231.46895034342 and parameters: {'lambda': 6.027423170320085e-06, 'alpha': 5.626587159863953e-09, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 10, 'max_depth': 10, 'random_state': 10, 'min_child_weight': 28}. Best is trial 84 with value: 3103.686250579581.


[18:06:10] ======== Monitor (0): Learner ========
[18:06:10] Configure: 0.001172s, 1 calls @ 1172us

[18:06:10] EvalOneIter: 0.000772s, 1 calls @ 772us

[18:06:10] GetGradient: 0.019993s, 1 calls @ 19993us

[18:06:10] PredictRaw: 5e-06s, 1 calls @ 5us

[18:06:10] UpdateOneIter: 0.750947s, 1 calls @ 750947us

[18:06:10] ======== Monitor (0): GBTree ========
[18:06:10] BoostNewTrees: 0.663481s, 1 calls @ 663481us

[18:06:10] CommitModel: 3e-06s, 1 calls @ 3us

[18:06:10] ======== Monitor (0): HistUpdater ========
[18:06:10] BuildHistogram: 0.174174s, 573 calls @ 174174us

[18:06:10] EvaluateSplits: 0.087785s, 673 calls @ 87785us

[18:06:10] InitData: 0.084558s, 100 calls @ 84558us

[18:06:10] InitRoot: 0.164155s, 100 calls @ 164155us

[18:06:10] LeafPartition: 2.1e-05s, 100 calls @ 21us

[18:06:10] UpdatePosition: 0.156648s, 573 calls @ 156648us

[18:06:10] UpdateTree: 0.629154s, 100 calls @ 629154us

[18:06:10] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 3


{'lambda': 3.709591043437966e-06,
 'alpha': 3.470264495040167e-08,
 'colsample_bytree': 0.9,
 'subsample': 0.7,
 'learning_rate': 1,
 'max_depth': 10,
 'random_state': 10,
 'min_child_weight': 6}

In [132]:
# put trials details in a dataframe:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,3937.530399,2023-11-06 18:05:12.575889,2023-11-06 18:05:12.681425,0 days 00:00:00.105536,1.195952e-10,0.9,4.080866e-05,1.00000,3,138,30,0.4,COMPLETE
1,1,5912.444335,2023-11-06 18:05:12.682424,2023-11-06 18:05:12.920700,0 days 00:00:00.238276,9.994301e-08,0.3,3.067967e-06,10.00000,5,68,10,0.8,COMPLETE
2,2,3957.955187,2023-11-06 18:05:12.923901,2023-11-06 18:05:13.626978,0 days 00:00:00.703077,2.608837e-08,0.3,5.969519e-09,0.00001,4,3,30,0.9,COMPLETE
3,3,3736.977683,2023-11-06 18:05:13.630399,2023-11-06 18:05:13.863696,0 days 00:00:00.233297,5.629740e-09,0.2,1.844228e-09,1.00000,11,53,30,0.4,COMPLETE
4,4,3957.955187,2023-11-06 18:05:13.869094,2023-11-06 18:05:13.987864,0 days 00:00:00.118770,1.029122e-09,0.2,1.759553e-08,10.00000,8,197,30,0.2,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,3131.480098,2023-11-06 18:06:04.213220,2023-11-06 18:06:06.279622,0 days 00:00:02.066402,1.751278e-08,0.9,4.999845e-06,1.00000,10,10,10,0.7,COMPLETE
96,96,3929.205927,2023-11-06 18:06:06.289365,2023-11-06 18:06:07.434858,0 days 00:00:01.145493,1.366711e-08,0.9,4.558912e-06,0.02000,10,11,10,0.7,COMPLETE
97,97,3957.473244,2023-11-06 18:06:07.436490,2023-11-06 18:06:08.656147,0 days 00:00:01.219657,1.755285e-08,0.9,1.539814e-06,0.00030,10,5,10,0.7,COMPLETE
98,98,3163.478835,2023-11-06 18:06:08.658383,2023-11-06 18:06:09.464439,0 days 00:00:00.806056,6.111546e-09,0.9,6.626607e-06,1.00000,10,9,10,0.7,COMPLETE


In [138]:
import plotly